# DS Lab Activity 1.1 (Part B). Creating and Preparing the DataFrames
--------------------------------------------------------------------------------------------------------

In this activity you will see common examples of data wrangling in pandas.

**Learning Outcomes**
1. be familiar with common data wrangling procedures in pandas
2. perform joins, groupby and other SQL-like procedures in pandas


To be able to use pandas you will need to import it into your code as follows:

In [1]:
import pandas as pd

## Creating a DataFrame Manually

DataFrames play an essential role in pandas, you can think of them as tables that are stored and manipulated efficiently using Python code.

Below we show a DataFrame for a set of values for the Tuberculosis (TB) disease in different countries for the years 2011-2013.

In [2]:
df = pd.DataFrame({
    'country':['FR','DE','US'],
    '2011':[7000,5800,15000],
    '2012':[6900,6000,14000],
    '2013':[7000,6200,13000]})


To view data we can simply call the DataFrame

In [3]:
df.head()

,country,2011,2012,2013
0,FR,7000,6900,7000
1,DE,5800,6000,6200
2,US,15000,14000,13000


We want to make sure that:
- each attribute (aka field) is a variable not a fixed reading 
- each row represents an independent record
- the table is expandable by adding records not fields (attributes)

The first criterion is satisfied by the 'country' attribute but not by '2011', '2012' or the '2013' attributes.
The third criterion is also not satisfied since moving to new years entail adding attributes instead of rows.
Therefore, we need to pre-process the data. Below we show some of the techniques that can be used to prepare the data. Note that this is not an extensive list of pre-processing techniques.


## Melting the Data

Melting the data involves converting some of the field names into actual values for the existing records and often result in expanding the data vertically and shrinking it horizontally.

If we look at the data above, we can realise that querying the country with TB for a specific year requires us to store the year programmatically because it is a name of the field and will make changing the code later more difficult.

A better structure would be to have the years stored as values instead of being field names. This way querying the dataframe and asking about a specific country and year can be handled by the user input rather than via the programmer. This particularly makes sense since the data is expected to change between a year and the other and adding fields again requires adjusting the structure of the dataset while adding a record is straightforward.

To do the above we need to use melt() to convert the attributes into data records

In [4]:
# here we are effectively saying to the melt function to keep the country 
# and to use it as an id for other records
pd.melt(df, id_vars ='country')


,country,variable,value
0,FR,2011,7000
1,DE,2011,5800
2,US,2011,15000
3,FR,2012,6900
4,DE,2012,6000
5,US,2012,14000
6,FR,2013,7000
7,DE,2013,6200
8,US,2013,13000


Note that pandas came up with the name for the two attributes that it created: 'variable' and 'value' to reflect what actually happen to the old data. Basically the old attributes become the 'variable' column and their values become the 'value' column.

A more elaborate version that allows us to explicitly specify the variable name of the attribute that we are adding along with a meaningful name for the actual values that we are dealing with.

In [5]:
pd.melt(df, id_vars ='country', var_name='year', value_name='count')

,country,year,count
0,FR,2011,7000
1,DE,2011,5800
2,US,2011,15000
3,FR,2012,6900
4,DE,2012,6000
5,US,2012,14000
6,FR,2013,7000
7,DE,2013,6200
8,US,2013,13000


## Pivoting the Data

Let us have a look at another DataFrame of a table that represents the amount of pollution for different cities (New York, London and Beijing).

In [6]:
df_pollution = pd.DataFrame({
    'city':['New York','New York','London','London','Beijing','Beijing'],
    'size':['large','small','large','small','large','small'],
    'amount':[23,14,22,16,121,56]})

df_pollution

,city,size,amount
0,New York,large,23
1,New York,small,14
2,London,large,22
3,London,small,16
4,Beijing,large,121
5,Beijing,small,56


This data needs to be converted into a more suitably formatted DataFrame to enable us to look into the data and filter it flexibly. Note that this is an opposite situation to the countries DataFrame. It makes more sense to have the large/small readings as a field since these are not expected to change and include them in the fabric of the dataset field themselves makes querying the data more effective. For example we could know immediately all the large particles pollution for all cities directly as follows

In [7]:
df_new = df_pollution.pivot(index='city',columns='size')

df_new

amount      
size      large small
city                 
Beijing     121    56
London       22    16
New York     23    14

# SQL Like Operations
The operations below are not written in SQL, rather they are similar to how we query a database using SQL. Be mindful, however, that DataFrames are not Databases, they are like data spreadsheets at runtime. See the following for more information about [DataFrames](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html#dataframe) in pandas.

## Joins (inner, left, right and outer)

If you are not familiar with the concept of joins, you may need to brush on this concept from an SQL context.

### Inner Joins

If we have two tables that have a common attribute (or a set of attributes) and both are talking about the same data, then we can join them into each other. The inner join is essentially the intersection between the two DataFrames, the left join is the left table along with the records from the right table that correspond with it (same for a right join but the other way around), while outer join is all of those records from both DataFrames. Let us see some examples

In [8]:
df1 = pd.DataFrame({
    'country':['FR','DE','US'],
    '2011':[7000,5800,15000],
    '2012':[6900,6000,14000],
    '2013':[7000,6200,13000]})
df1

,country,2011,2012,2013
0,FR,7000,6900,7000
1,DE,5800,6000,6200
2,US,15000,14000,13000


In [9]:
df2 = pd.DataFrame({
    'cntry':['FR','DE','USSR'],
    '2014':[6500,5900,15500],
    '2015':[7000,7000,15000],
    '2016':[8000,6500,13300]})
df2

,cntry,2014,2015,2016
0,FR,6500,7000,8000
1,DE,5900,7000,6500
2,USSR,15500,15000,13300


Then both can be inner-joined as follows:
first we need to make sure that both have the same field. In this case, both have a field that represents the country but in the df2 it is spelled differently. Hence, we need to rename it in df2 as in df1. To do so we execute:

In [10]:
#df2['country'] = df2['cntry'] # careful executing this line will add an extra column to your DataFrame
df2.rename(columns = {'cntry':'country'}, inplace = True)
df2

,country,2014,2015,2016
0,FR,6500,7000,8000
1,DE,5900,7000,6500
2,USSR,15500,15000,13300


Ok so now we are ready to join the two DataFrames in one according to the common 'country' field:

In [11]:
inner = pd.merge(df1,df2, on='country')
inner

,country,2011,2012,2013,2014,2015,2016
0,FR,7000,6900,7000,6500,7000,8000
1,DE,5800,6000,6200,5900,7000,6500


Note that we only have the records that have the same country.

### Left Join

In [12]:
left = pd.merge(df1, df2, on = 'country', how = 'left')
left

,country,2011,2012,2013,2014,2015,2016
0,FR,7000,6900,7000,6500.0,7000.0,8000.0
1,DE,5800,6000,6200,5900.0,7000.0,6500.0
2,US,15000,14000,13000,NaN,NaN,NaN


Note that we have all the records from df1 and the corresponding missing data for df2 has been filled with NaN

### Right Join

In [13]:
right = pd.merge(df1, df2, on = 'country', how = 'right')
right

,country,2011,2012,2013,2014,2015,2016
0,FR,7000.0,6900.0,7000.0,6500,7000,8000
1,DE,5800.0,6000.0,6200.0,5900,7000,6500
2,USSR,NaN,NaN,NaN,15500,15000,13300


Note that we have all the records from df2 and the corresponding missing data for df1 has been filled with NaN

### Outer Join

In [14]:
outer = pd.merge(df1, df2, on = 'country', how = 'outer')
outer

,country,2011,2012,2013,2014,2015,2016
0,FR,7000.0,6900.0,7000.0,6500.0,7000.0,8000.0
1,DE,5800.0,6000.0,6200.0,5900.0,7000.0,6500.0
2,US,15000.0,14000.0,13000.0,NaN,NaN,NaN
3,USSR,NaN,NaN,NaN,15500.0,15000.0,13300.0


Note that we have all the records from df1 and df2 and the corresponding missing data for df2 and df1 has been filled with NaN

Now we can melt the data into the shape that we want (please be mindful that if you run the same code in its place multiple times it will add up to the same DataFrame, if you want to make sure you get the correct results go to the Jupyter menu above and select 'Cell' -> 'Run All Above'

In [15]:
#inner = inner.melt(id_vars ='country')
inner = inner.melt(id_vars ='country', var_name='year', value_name='count')
inner

,country,year,count
0,FR,2011,7000
1,DE,2011,5800
2,FR,2012,6900
3,DE,2012,6000
4,FR,2013,7000
5,DE,2013,6200
6,FR,2014,6500
7,DE,2014,5900
8,FR,2015,7000
9,DE,2015,7000


In [16]:
left = left.melt(id_vars ='country')
left

#right = right.melt(id_vars ='country')
#right

#outer = outer.melt(id_vars ='country')
#outer


,country,variable,value
0,FR,2011,7000.0
1,DE,2011,5800.0
2,US,2011,15000.0
3,FR,2012,6900.0
4,DE,2012,6000.0
5,US,2012,14000.0
6,FR,2013,7000.0
7,DE,2013,6200.0
8,US,2013,13000.0
9,FR,2014,6500.0
